I copied the data into a .csv file and imported it into the IBM Cloud where I used the below code to create a dataframe from the information. The dataframe is called simply df_data_1

In [2]:
# The code was removed by Watson Studio for sharing.

,M1A,Not assigned,Not assigned.1
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [4]:
#Renaming the columns per the instructions
df_data_1.columns = ['Post Code', 'Borough', 'Neighborhood']

In [5]:
#Getting rid of data with no assigned borough
df  = df_data_1[df_data_1['Borough']!= 'Not assigned']

The shape of the initial data frame is 103 rows and 3 columns. 

In [6]:
df.shape

(103, 3)

In [7]:
df.head()

,Post Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


I could not get the geocoder to work, so I also imported the .csv of the lats and longs and put them in a data frame. I joined the 2 together to make one dataframe of neighborhood data. 

In [13]:
#Importing the csv of the latitudes and longitudes 
body = client_dd97143e82fe4301884bf813f9b40bfc.get_object(Bucket='datasciencecapstone-donotdelete-pr-uhkcxbgktbi4er',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()
df_data_2.columns = ['Post Code', 'Latitude', 'Longitude']

In [18]:
#combining the dataframes to add the lat and long coordinates for the post codes. 
df2 = df.join(df_data_2.set_index('Post Code'), on='Post Code')
df2.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M4A,North York,Victoria Village,43.725882,-79.315572
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


The next several cells were just getting all the libraries and packages installed and imported for the maps and clustering. 

In [42]:
#installing folium to make the maps
pip install folium
pip install geocoder
import geocoder

SyntaxError: invalid syntax (<ipython-input-42-46dc7dead5cd>, line 3)

In [43]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import geocoder

print('Libraries imported.')

Libraries imported.


In [44]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

From here, I mapped the Toronto neighborhoods as a first pass. 

In [47]:
#find the lat and long of Toronto to center the map
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'Toronto_Lab')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [54]:

# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{},{}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

I decided to analyze only the neighborhoods in Downtown Toronto. So I sorted out a data frame that only contained those neighborhoods. 

In [57]:
#isolating the boroughs that are in downtown toronto
df3  = df2[df2['Borough']== 'Downtown Toronto']
df3

,Post Code,Borough,Neighborhood,Latitude,Longitude
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
12,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
21,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
30,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
39,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
40,M6G,Downtown Toronto,Christie,43.669542,-79.422564
48,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
57,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
66,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


I initialized Foursquare data and then used the code from the lab to get venue data for the neighborhoods I'm focusing on. 

In [95]:
#initializing FourSquare information
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [62]:
#using the lab function to get the venues in all the downtown toronto boroughs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
downtown_tor_venues = getNearbyVenues(names = df3['Neighborhood'], latitudes = df3['Latitude'], longitudes = df3['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [63]:
downtown_tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


I then counted the number of venues in the target neighborhoods (up to the limit of 100) 

In [86]:
#counting the number of values
venue_number = downtown_tor_venues.groupby('Neighborhood').count()

In [87]:
#dropping the columns that are repeated. 
venue_number.sort_values(by=['Venue'])
venue_number = venue_number.drop(columns=['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'])

In [88]:
#resetting the index on the count dataframe
venue_number = venue_number.reset_index()
venue_number

,Neighborhood,Venue
0,Berczy Park,59
1,"CN Tower, King and Spadina, Railway Lands, Har...",15
2,Central Bay Street,63
3,Christie,16
4,Church and Wellesley,84
5,"Commerce Court, Victoria Hotel",100
6,"First Canadian Place, Underground city",100
7,"Garden District, Ryerson",100
8,"Harbourfront East, Union Station, Toronto Islands",100
9,"Kensington Market, Chinatown, Grange Park",65


Then I used the standard clustering code to cluster the neighborhoods and then add the cluster labels to the data frame. 

In [89]:
# set number of clusters
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_number)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

ValueError: could not convert string to float: 'University of Toronto, Harbord'

In [90]:
venue_number.insert(0, 'Cluster Labels', kmeans.labels_)

Then I joined the count and cluster labels data frame with the locational data to be able to map the clusters. 

In [92]:
venue_number2 = df3.join(venue_number.set_index('Neighborhood'),on = 'Neighborhood')
venue_number2

,Post Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Venue
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,45
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,35
12,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,100
21,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,79
30,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,59
39,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,63
40,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,16
48,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,96
57,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,1,100
66,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,1,100


The last step is to map the clusters. The visualization I am trying to achieve is a map of neighborhoods with more venues. It looks pretty good! 

In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venue_number2['Latitude'], venue_number2['Longitude'], venue_number2['Neighborhood'], venue_number2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters